# File created the 17 12 2021 in order to compute the real value of differents metrics values used in the IEEEAccess paper

- 2.0.0 ==> Synthetic 5 (TwoD2_V2_0_0())
- 3.3.0 ==> Synthetic 1 (TwoD2_V3_3_0())
- 3.4.0 ==> Synthetic 2 (TwoD2_V3_4_0())
- 4.3.0 ==> Synthetic 6 (TwoD2_V4_3_0())
- 4.3.1 ==> Synthetic 3 (TwoD2_V4_3_1())
- 4.3.2 ==> Synthetic 4 (TwoD2_V4_3_2())

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

In [6]:
import sys
#path_top="../../../../../../../"
path_top = "../"
sys.path.append(path_top)
from datasets import datasets as datat
dts = datat.synthetic_datasets(path_top=path_top)

util = datat.utilitaries()

from metrics import useful_infos as ue

from metrics import performances
perf = performances.performances()

In [137]:
def MetricsFromYClassifiedData(data_path, dataset_name, method_name) :
    print("################### "+method_name +" on "+ dataset_name+" ##############################")
    # Load the dataset
    dataset = datat.load_data_from_CSV(data_path);
    # Split in execution results
    XBrut, YBrut, YClassified, Scores, PathLengths = util.split_in_results(dataset)
    
    #print(YBrut) 
    #print(YClassified)
    print("Scikit-learn used confusion matrix details")
    print("    tn  fp")
    print("    fn  tp")
    print("          ")
    tn, fp, fn, tp, cm, auc, spec, prec, rec, f1, far = perf.performance_summary(Y_predict=YClassified, Y_original=YBrut, 
                                                                                     scores=Scores, CPUTime = None, 
                                                                                     UseMemory=None, print_result=True, 
                                                                                     outputPRAUC = False, labels = [1,-1],
                                                                                     pos_label=-1)
    print("tn = "+str(tn)+", fp = "+str(fp)+", fn = "+str(fn)+", tp = "+str(tp))
    print("Recall : " + str(tp/(tp+fn)))
    print("Specificity : " + str(tn/(tn+fp)))
    print("False alarm rate (%) : " + str(fp * 100/(fp+tn)))
    print("f1_score : " + str(2*((tp/(tp+fp) * tp/(tp+fn))/(tp/(tp+fp) + tp/(tp+fn)))))
    print("Precision : " + str(tp/(tp+fp)))
    print("")

In [87]:
def MetricsFromOldConfusionMatrix(data_path, dataset_name, method_name) :
    # Load the dataset
    dataset = datat.load_data_from_CSV(data_path);
    #ROC AUC,Recall,Specificity,FAR (%),Precision,F1 Score,TN,FP,FN,TP
    #We have to replace Negative by Positive in order to have abnormal as positive and normal as negative
    #That means TN = TP, FP = FN, FN = FP and TP = TN
    tn = int(dataset[9])
    fp = int(dataset[8])
    fn = int(dataset[7])
    tp = int(dataset[6])
    #cm = dataset[]  
    auc  = float(dataset[0])
    spec = float(dataset[2])
    prec = float(dataset[4])
    rec  = float(dataset[1])
    f1   = float(dataset[5])
    far  = float(dataset[3])
    #print(dataset)  
    
    print("################### OLD Metrics Values ##############################")
    print("tn = "+str(tp)+", fp = "+str(fn)+", fn = "+str(fp)+", tp = "+str(tn))
    print("ROC AUC : " + str(auc))
    print("Specificity : " + str(spec))
    print("Precision : " + str(prec))
    print("Recall : " + str(rec))
    print("f1_score : " + str(f1))
    print("False alarm rate (%) : " + str(far))
    print("We have to replace Negative by Positive in order to have abnormal as positive and normal as negative "+ 
          "That means TN = TP, FP = FN, FN = FP and TP = TN")
    
    print("################### NEW Metrics Values ##############################")
    print("tn = "+str(tn)+", fp = "+str(fp)+", fn = "+str(fn)+", tp = "+str(tp))
    print("Specificity : " + str(tn/(tn+fp)))
    print("Precision : " + str(tp/(tp+fp)))
    print("Recall : " + str(tp/(tp+fn)))
    print("f1_score : " + str(2*((tp/(tp+fp) * tp/(tp+fn))/(tp/(tp+fp) + tp/(tp+fn)))))
    print("False alarm rate (%) : " + str(fp * 100/(fp+tn)))

In [125]:
def MetricsFromDistinctConfusionMatrix(tns, fps, fns, tps, dataset_name, method_name) :
    fars = []
    f1s = []
    specs = []
    recs = []
    precs = []
    for i in range(0, len(tns), 1):
        tn = tns[i]
        fp = fps[i]
        fn = fns[i]
        tp = tps[i]
        specs = np.append(specs, tn/(tn+fp))
        precs = np.append(precs, tp/(tp+fp))
        recs  = np.append(recs, tp/(tp+fn))
        f1s   = np.append(f1s, 2*((tp/(tp+fp) * tp/(tp+fn))/(tp/(tp+fp) + tp/(tp+fn))))
        fars  = np.append(fars, fp * 100/(fp+tn))

    print("################### NEW Metrics Values for "+method_name +" on "+ dataset_name+" ##############################")
    print("tn = "+str(tns)+", fp = "+str(fps)+", fn = "+str(fns)+", tp = "+str(tps))
    print("Recall : " + str(round(np.mean(recs), 2)))
    print("Specificity : " + str(round(np.mean(specs), 2)))
    print("False alarm rate (%) : " + str(round(np.mean(fars), 2)))
    print("f1_score : " + str(round(np.mean(f1s), 2)))
    print("Precision : " + str(round(np.mean(precs), 2)))
    

In [126]:
# Config
data_folder = "usedData/"

### FOREST VS EXTENDED IFOREST VS MVIFOREST ON SYNTHETIC DATASETS
##### Synthetic\_1 = 3.3.0 (IF = 1, MVIF = 0, EIF = 2)
##### Synthetic\_2 = 3.4.0 (IF = 2, MVIF = 4, EIF = 0)
##### Synthetic\_3 = 4.3.1 (IF = 2, MVIF = 0, EIF = 3)
##### Synthetic\_4 = 4.3.2 (IF = 3, MVIF = 1, EIF = 4)
##### Synthetic\_5 = 2.0.0 (IF = 1, MVIF = 0, EIF = 4)

In [138]:
dataset_name = "Synthetic_1 = 3.3.0 (IF = 1, MVIF = 0, EIF = 2)"

method_name = "MVIF"
data_path = data_folder+"Exec_0_Experiment_MVIForest_TwoD_V3.3.0_DataResults_2021-10-19-17-12-44-469662.csv"
MetricsFromYClassifiedData(data_path, dataset_name, method_name)
method_name = "IF"
data_path = data_folder+"Exec_1_Experiment_IForest_TwoD_V3.3.0_DataResults_2021-10-19-17-11-54-738531.csv"
MetricsFromYClassifiedData(data_path, dataset_name, method_name)
method_name = "EIF"
data_path = data_folder+"Exec_2_Experiment_Local EIF_TwoD_V3.3.0_DataResults_2021-10-19-17-15-02-624582.csv"
MetricsFromYClassifiedData(data_path, dataset_name, method_name)

################### MVIF on Synthetic_1 = 3.3.0 (IF = 1, MVIF = 0, EIF = 2) ##############################
Scikit-learn used confusion matrix details
    tn  fp
    fn  tp
          
Confusion matrice
[[1410   90]
 [   0   15]]
ROC AUC : 0.97
Specificity : 0.94
Precision : 0.14285714285714285
Recall : 1.0
f1_score : 0.25
PR AUC : 0.009900990099009901
False alarm rate (%) : 6.000000000000005
CPU Time (s) : None
Memory Consumption (Mo) : None
****************************************************************
tn = 1410, fp = 90, fn = 0, tp = 15
Recall : 1.0
Specificity : 0.94
False alarm rate (%) : 6.0
f1_score : 0.25
Precision : 0.14285714285714285

################### IF on Synthetic_1 = 3.3.0 (IF = 1, MVIF = 0, EIF = 2) ##############################
Scikit-learn used confusion matrix details
    tn  fp
    fn  tp
          
Confusion matrice
[[1400  100]
 [   0   15]]
ROC AUC : 0.9666666666666667
Specificity : 0.9333333333333333
Precision : 0.13043478260869565
Recall : 1.0
f1_score : 0.

In [139]:
dataset_name = "Synthetic_2 = 3.4.0 (IF = 2, MVIF = 4, EIF = 0)"

method_name = "MVIF"
data_path = data_folder+"Exec_4_Experiment_MVIForest_TwoD_V3.4.0_DataResults_2021-10-19-17-19-18-149729.csv"
MetricsFromYClassifiedData(data_path, dataset_name, method_name)
method_name = "IF"
data_path = data_folder+"Exec_2_Experiment_IForest_TwoD_V3.4.0_DataResults_2021-10-19-17-17-51-974985.csv"
MetricsFromYClassifiedData(data_path, dataset_name, method_name)
method_name = "EIF"
data_path = data_folder+"Exec_0_Experiment_Local EIF_TwoD_V3.4.0_DataResults_2021-10-19-17-19-46-510496.csv"
MetricsFromYClassifiedData(data_path, dataset_name, method_name)

################### MVIF on Synthetic_2 = 3.4.0 (IF = 2, MVIF = 4, EIF = 0) ##############################
Scikit-learn used confusion matrix details
    tn  fp
    fn  tp
          
Confusion matrice
[[1397  103]
 [   3   12]]
ROC AUC : 0.8656666666666668
Specificity : 0.9313333333333333
Precision : 0.10434782608695652
Recall : 0.8
f1_score : 0.18461538461538463
PR AUC : 0.00834936350777935
False alarm rate (%) : 6.866666666666665
CPU Time (s) : None
Memory Consumption (Mo) : None
****************************************************************
tn = 1397, fp = 103, fn = 3, tp = 12
Recall : 0.8
Specificity : 0.9313333333333333
False alarm rate (%) : 6.866666666666666
f1_score : 0.1846153846153846
Precision : 0.10434782608695652

################### IF on Synthetic_2 = 3.4.0 (IF = 2, MVIF = 4, EIF = 0) ##############################
Scikit-learn used confusion matrix details
    tn  fp
    fn  tp
          
Confusion matrice
[[1386  114]
 [  12    3]]
ROC AUC : 0.562
Specificity : 0.924

/Users/togbe/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
<ipython-input-137-915a5b429d0c>:23: RuntimeWarning: invalid value encountered in long_scalars
  print("f1_score : " + str(2*((tp/(tp+fp) * tp/(tp+fn))/(tp/(tp+fp) + tp/(tp+fn)))))
<ipython-input-137-915a5b429d0c>:24: RuntimeWarning: invalid value encountered in long_scalars
  print("Precision : " + str(tp/(tp+fp)))


In [140]:
dataset_name = "Synthetic_3 = 4.3.1 (IF = 2, MVIF = 0, EIF = 3)"

method_name = "MVIF"
data_path = data_folder+"Exec_0_Experiment_MVIForest_TwoD_V4.3.1_DataResults_2021-10-19-17-30-24-453641.csv"
MetricsFromYClassifiedData(data_path, dataset_name, method_name)
method_name = "IF"
data_path = data_folder+"Exec_2_Experiment_IForest_TwoD_V4.3.1_DataResults_2021-10-19-17-29-45-100442.csv"
MetricsFromYClassifiedData(data_path, dataset_name, method_name)
method_name = "EIF"
data_path = data_folder+"Exec_3_Experiment_Local EIF_TwoD_V4.3.1_DataResults_2021-10-19-17-33-15-115077.csv"
MetricsFromYClassifiedData(data_path, dataset_name, method_name)

################### MVIF on Synthetic_3 = 4.3.1 (IF = 2, MVIF = 0, EIF = 3) ##############################
Scikit-learn used confusion matrix details
    tn  fp
    fn  tp
          
Confusion matrice
[[1414   86]
 [   0   15]]
ROC AUC : 0.9713333333333334
Specificity : 0.9426666666666667
Precision : 0.1485148514851485
Recall : 1.0
f1_score : 0.25862068965517243
PR AUC : 0.009900990099009901
False alarm rate (%) : 5.733333333333334
CPU Time (s) : None
Memory Consumption (Mo) : None
****************************************************************
tn = 1414, fp = 86, fn = 0, tp = 15
Recall : 1.0
Specificity : 0.9426666666666667
False alarm rate (%) : 5.733333333333333
f1_score : 0.25862068965517243
Precision : 0.1485148514851485

################### IF on Synthetic_3 = 4.3.1 (IF = 2, MVIF = 0, EIF = 3) ##############################
Scikit-learn used confusion matrix details
    tn  fp
    fn  tp
          
Confusion matrice
[[1384  116]
 [   0   15]]
ROC AUC : 0.9613333333333334
Specifi

In [141]:
dataset_name = "Synthetic_4 = 4.3.2 (IF = 3, MVIF = 1, EIF = 4)"

method_name = "MVIF"
data_path = data_folder+"Exec_1_Experiment_MVIForest_TwoD_V4.3.2_DataResults_2021-10-19-17-36-52-890393.csv"
MetricsFromYClassifiedData(data_path, dataset_name, method_name)
method_name = "IF"
data_path = data_folder+"Exec_3_Experiment_IForest_TwoD_V4.3.2_DataResults_2021-10-19-17-36-03-337399.csv"
MetricsFromYClassifiedData(data_path, dataset_name, method_name)
method_name = "EIF"
data_path = data_folder+"Exec_4_Experiment_Local EIF_TwoD_V4.3.2_DataResults_2021-10-19-17-40-10-861780.csv"
MetricsFromYClassifiedData(data_path, dataset_name, method_name)

################### MVIF on Synthetic_4 = 4.3.2 (IF = 3, MVIF = 1, EIF = 4) ##############################
Scikit-learn used confusion matrix details
    tn  fp
    fn  tp
          
Confusion matrice
[[1419   81]
 [   0   15]]
ROC AUC : 0.973
Specificity : 0.946
Precision : 0.15625
Recall : 1.0
f1_score : 0.2702702702702703
PR AUC : 0.009900990099009901
False alarm rate (%) : 5.400000000000005
CPU Time (s) : None
Memory Consumption (Mo) : None
****************************************************************
tn = 1419, fp = 81, fn = 0, tp = 15
Recall : 1.0
Specificity : 0.946
False alarm rate (%) : 5.4
f1_score : 0.2702702702702703
Precision : 0.15625

################### IF on Synthetic_4 = 4.3.2 (IF = 3, MVIF = 1, EIF = 4) ##############################
Scikit-learn used confusion matrix details
    tn  fp
    fn  tp
          
Confusion matrice
[[1381  119]
 [   0   15]]
ROC AUC : 0.9603333333333333
Specificity : 0.9206666666666666
Precision : 0.11194029850746269
Recall : 1.0
f1_sco

In [142]:
dataset_name = "Synthetic_5 = 2.0.0 (IF = 1, MVIF = 0, EIF = 4)"

method_name = "MVIF"
data_path = data_folder+"Exec_0_Experiment_MVIForest_TwoD_V2.0.0_DataResults_2021-10-19-17-06-51-417427.csv"
MetricsFromYClassifiedData(data_path, dataset_name, method_name)
method_name = "IF"
data_path = data_folder+"Exec_1_Experiment_IForest_TwoD_V2.0.0_DataResults_2021-10-19-17-05-58-120613.csv"
MetricsFromYClassifiedData(data_path, dataset_name, method_name)
method_name = "EIF"
data_path = data_folder+"Exec_4_Experiment_Local EIF_TwoD_V2.0.0_DataResults_2021-10-19-17-10-21-839804.csv"
MetricsFromYClassifiedData(data_path, dataset_name, method_name)

################### MVIF on Synthetic_5 = 2.0.0 (IF = 1, MVIF = 0, EIF = 4) ##############################
Scikit-learn used confusion matrix details
    tn  fp
    fn  tp
          
Confusion matrice
[[1351  149]
 [  15    0]]
ROC AUC : 0.4503333333333333
Specificity : 0.9006666666666666
Precision : 0.0
Recall : 0.0
f1_score : 0.0
PR AUC : 0.010980966325036604
False alarm rate (%) : 9.933333333333339
CPU Time (s) : None
Memory Consumption (Mo) : None
****************************************************************
tn = 1351, fp = 149, fn = 15, tp = 0
Recall : 0.0
Specificity : 0.9006666666666666
False alarm rate (%) : 9.933333333333334
f1_score : nan
Precision : 0.0

################### IF on Synthetic_5 = 2.0.0 (IF = 1, MVIF = 0, EIF = 4) ##############################
Scikit-learn used confusion matrix details
    tn  fp
    fn  tp
          
Confusion matrice
[[1349  151]
 [  15    0]]
ROC AUC : 0.44966666666666666
Specificity : 0.8993333333333333
Precision : 0.0
Recall : 0.0
f1_sc

<ipython-input-137-915a5b429d0c>:23: RuntimeWarning: invalid value encountered in double_scalars
  print("f1_score : " + str(2*((tp/(tp+fp) * tp/(tp+fn))/(tp/(tp+fp) + tp/(tp+fn)))))
<ipython-input-137-915a5b429d0c>:23: RuntimeWarning: invalid value encountered in double_scalars
  print("f1_score : " + str(2*((tp/(tp+fp) * tp/(tp+fn))/(tp/(tp+fp) + tp/(tp+fn)))))
/Users/togbe/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
<ipython-input-137-915a5b429d0c>:23: RuntimeWarning: invalid value encountered in long_scalars
  print("f1_score : " + str(2*((tp/(tp+fp) * tp/(tp+fn))/(tp/(tp+fp) + tp/(tp+fn)))))
<ipython-input-137-915a5b429d0c>:24: RuntimeWarning: invalid value encountered in long_scalars
  print("Precision : " + str(tp/(tp+fp)))


### EVALUATION OF MVIFOREST ON REAL DATASETS

In [127]:
dataset_name = "Shuttle"

method_name = "MVIForest"
tps = [3449, 3443, 3447, 3457, 3437]
fns = [62, 68, 64, 54, 74]
fps = [1057, 957, 1165, 1283, 917]
tns = [44529, 44629, 44421, 44303, 44669]
MetricsFromDistinctConfusionMatrix(tns = tns, fps = fps, fns = fns, tps = tps, 
                                   dataset_name = dataset_name, method_name = method_name)

method_name = "IForest"
tps = [3455, 3462, 3477, 3468, 3462]
fns = [56, 49, 34, 43, 49]
fps = [1171, 1095, 1701, 1529, 1314]
tns = [44415, 44491, 44057, 44272]
MetricsFromDistinctConfusionMatrix(tns = tns, fps = fps, fns = fns, tps = tps, 
                                   dataset_name = dataset_name, method_name = method_name)

################### NEW Metrics Values for MVIForest on Shuttle ##############################
tn = [44529, 44629, 44421, 44303, 44669], fp = [1057, 957, 1165, 1283, 917], fn = [62, 68, 64, 54, 74], tp = [3449, 3443, 3447, 3457, 3437]
Recall : 0.98
Specificity : 0.98
False alarm rate (%) : 2.36
f1_score : 0.86
Precision : 0.76
################### NEW Metrics Values for IForest on Shuttle ##############################
tn = [44415, 44491, 44057, 44272], fp = [1171, 1095, 1701, 1529, 1314], fn = [56, 49, 34, 43, 49], tp = [3455, 3462, 3477, 3468, 3462]
Recall : 0.99
Specificity : 0.97
False alarm rate (%) : 3.01
f1_score : 0.83
Precision : 0.72


In [128]:
dataset_name = "SMTP"

method_name = "MVIForest"
tps = [23, 23, 22, 23, 22]
fns = [7, 7, 8, 7, 8]
fps = [7188, 6663, 7814, 7753, 7709]
tns = [87938, 88463, 87312, 87373, 87417]
MetricsFromDistinctConfusionMatrix(tns = tns, fps = fps, fns = fns, tps = tps, 
                                   dataset_name = dataset_name, method_name = method_name)

method_name = "IForest"
tps = [22, 22, 8, 23, 22, 22]
fns = [8, 8, 7, 8, 8]
fps = [6364, 6484, 7706, 7287, 6573]
tns = [88762, 88642, 87420, 87839, 88553]
MetricsFromDistinctConfusionMatrix(tns = tns, fps = fps, fns = fns, tps = tps, 
                                   dataset_name = dataset_name, method_name = method_name)

################### NEW Metrics Values for MVIForest on SMTP ##############################
tn = [87938, 88463, 87312, 87373, 87417], fp = [7188, 6663, 7814, 7753, 7709], fn = [7, 7, 8, 7, 8], tp = [23, 23, 22, 23, 22]
Recall : 0.75
Specificity : 0.92
False alarm rate (%) : 7.81
f1_score : 0.01
Precision : 0.0
################### NEW Metrics Values for IForest on SMTP ##############################
tn = [88762, 88642, 87420, 87839, 88553], fp = [6364, 6484, 7706, 7287, 6573], fn = [8, 8, 7, 8, 8], tp = [22, 22, 8, 23, 22, 22]
Recall : 0.7
Specificity : 0.93
False alarm rate (%) : 7.24
f1_score : 0.01
Precision : 0.0


In [129]:
dataset_name = "ForestCover"

method_name = "MVIForest"
tps = [2297, 1652, 1951, 1808, 1894]
fns = [450, 1095, 796, 939, 853]
fps = [28189, 24953, 30753, 29417, 25756]
tns = [255112, 258348, 252548, 253884, 257545]
MetricsFromDistinctConfusionMatrix(tns = tns, fps = fps, fns = fns, tps = tps, 
                                   dataset_name = dataset_name, method_name = method_name)

method_name = "IForest"
tps = [2491, 2503, 2286, 2621, 2050]
fns = [256, 244, 461, 126, 697]
fps = [46018, 47202, 41288, 39194, 46369]
tns = [237283, 236099, 242013, 244107, 236932]
MetricsFromDistinctConfusionMatrix(tns = tns, fps = fps, fns = fns, tps = tps, 
                                   dataset_name = dataset_name, method_name = method_name)

################### NEW Metrics Values for MVIForest on ForestCover ##############################
tn = [255112, 258348, 252548, 253884, 257545], fp = [28189, 24953, 30753, 29417, 25756], fn = [450, 1095, 796, 939, 853], tp = [2297, 1652, 1951, 1808, 1894]
Recall : 0.7
Specificity : 0.9
False alarm rate (%) : 9.82
f1_score : 0.12
Precision : 0.06
################### NEW Metrics Values for IForest on ForestCover ##############################
tn = [237283, 236099, 242013, 244107, 236932], fp = [46018, 47202, 41288, 39194, 46369], fn = [256, 244, 461, 126, 697], tp = [2491, 2503, 2286, 2621, 2050]
Recall : 0.87
Specificity : 0.84
False alarm rate (%) : 15.54
f1_score : 0.1
Precision : 0.05


In [131]:
dataset_name = "HTTP"

method_name = "MVIForest"
tps = [2209, 2211, 2211, 2211, 2211]
fns = [2, 0, 0, 0, 0]
fps = [11873, 13792, 10095, 11249, 14401]
tns = [553413, 551494, 555191, 554037, 550885]
MetricsFromDistinctConfusionMatrix(tns = tns, fps = fps, fns = fns, tps = tps, 
                                   dataset_name = dataset_name, method_name = method_name)

method_name = "IForest"
tps = [2211, 2211, 2211, 2211, 2211]
fns = [0, 0, 0, 0, 0]
fps = [16509, 12195, 14516, 13325, 12179]
tns = [548777, 553091, 550770, 551961, 553107]
MetricsFromDistinctConfusionMatrix(tns = tns, fps = fps, fns = fns, tps = tps, 
                                   dataset_name = dataset_name, method_name = method_name)

################### NEW Metrics Values for MVIForest on HTTP ##############################
tn = [553413, 551494, 555191, 554037, 550885], fp = [11873, 13792, 10095, 11249, 14401], fn = [2, 0, 0, 0, 0], tp = [2209, 2211, 2211, 2211, 2211]
Recall : 1.0
Specificity : 0.98
False alarm rate (%) : 2.17
f1_score : 0.27
Precision : 0.15
################### NEW Metrics Values for IForest on HTTP ##############################
tn = [548777, 553091, 550770, 551961, 553107], fp = [16509, 12195, 14516, 13325, 12179], fn = [0, 0, 0, 0, 0], tp = [2211, 2211, 2211, 2211, 2211]
Recall : 1.0
Specificity : 0.98
False alarm rate (%) : 2.43
f1_score : 0.25
Precision : 0.14


### IMPACT OF THE PRESENCE OF ABNORMAL DATA IN THE SAMPLE

In [89]:
method_name = "IForest"
dataset_name = "4.3.1 - AllDataInTrain"
data_path = data_folder+"Experiment_IForest_NormalOnly_TwoD_V4.3.1_2020-10-12-14-07-10-697370.csv"
MetricsFromOldConfusionMatrix(data_path, dataset_name, method_name)

################### OLD Metrics Values ##############################
tn = 15, fp = 0, fn = 116, tp = 1384
ROC AUC : 0.9613333333333334
Specificity : 1.0
Precision : 1.0
Recall : 0.9226666666666666
f1_score : 0.9597780859916781
False alarm rate (%) : 88.54961832061069
We have to replace Negative by Positive in order to have abnormal as positive and normal as negative That means TN = TP, FP = FN, FN = FP and TP = TN
################### NEW Metrics Values ##############################
tn = 1384, fp = 116, fn = 0, tp = 15
Specificity : 0.9226666666666666
Precision : 0.11450381679389313
Recall : 1.0
f1_score : 0.2054794520547945
False alarm rate (%) : 7.733333333333333


In [90]:
method_name = "IForest"
dataset_name = "4.3.1 - NormalDataOnlyInTrain"
data_path = data_folder+"Experiment_IForest_NormalOnly_TwoD_V4.3.1_2020-10-12-14-07-02-715333.csv"
MetricsFromOldConfusionMatrix(data_path, dataset_name, method_name)

################### OLD Metrics Values ##############################
tn = 15, fp = 0, fn = 132, tp = 1368
ROC AUC : 0.956
Specificity : 1.0
Precision : 1.0
Recall : 0.912
f1_score : 0.9539748953974896
False alarm rate (%) : 89.79591836734694
We have to replace Negative by Positive in order to have abnormal as positive and normal as negative That means TN = TP, FP = FN, FN = FP and TP = TN
################### NEW Metrics Values ##############################
tn = 1368, fp = 132, fn = 0, tp = 15
Specificity : 0.912
Precision : 0.10204081632653061
Recall : 1.0
f1_score : 0.18518518518518523
False alarm rate (%) : 8.8
